In [31]:
!pip install biopython

In [32]:
import os, gc, re, csv
from collections import defaultdict

import pandas as pd
from tqdm.auto import tqdm
import numpy as np

from collections import Counter
import math

from Bio import SeqIO

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

from sklearn.multiclass import OneVsRestClassifier

In [33]:
ROOT_DIR = './'
is_kaggle = True
if is_kaggle:
    ROOT_DIR = '/kaggle/input/cafa-6-protein-function-prediction/'

In [34]:
terms_df = pd.read_csv(os.path.join(ROOT_DIR + "Train/train_terms.tsv"), sep="\t", usecols=["EntryID", "term"])
train_annotations = terms_df.groupby("EntryID")["term"].apply(list).to_dict()

train_sq = []
train_answer = []

terms_to_answer = terms_df.groupby('EntryID')['term'].apply(list).to_dict()

cnt = 0


for record in SeqIO.parse(os.path.join(ROOT_DIR + "Train/train_sequences.fasta"), "fasta"):
    try:
        if "|" in record.id:
            clean_id = record.id.split("|")[1]
        else:
            clean_id = record.id
        a = record.description.split("OX=")
        b = a[1].split(" ")[0]
        if clean_id:
            train_sq.append({
                "id": clean_id,
                "tax": b,
                "seq": str(record.seq),
                "answer": terms_to_answer[clean_id]
            })
        else:
            print("123")
    except IndexError:
        continue

test_sq = []

for record in SeqIO.parse(os.path.join(ROOT_DIR + "Test/testsuperset.fasta"), "fasta"):
    tax = record.description.split(" ")[1]
    # if (tax not in test_gr):
    #     test_gr[tax] = []
    test_sq.append({
        "id": record.id,
        "tax": tax,
        "seq": str(record.seq)
    })

test_df = pd.DataFrame(test_sq)
train_df = pd.DataFrame(train_sq)

In [35]:
go_counter = Counter()
for terms in train_df["answer"]:
    go_counter.update(terms)

# Lấy top-100
TOP_K = 300
topK = set([go for go, _ in go_counter.most_common(TOP_K)])

# print("Top 100 GO:", top100[:10])

In [36]:
def extract_sequence_features_tf_idf(
    seq,
    idf_aa=None,
    idf_di=None,
    idf_tri=None
):
    """
    seq      : chuỗi protein đầu vào
    idf_*    : dict chứa IDF tương ứng. Nếu None → không áp dụng IDF (giữ tf thuần)
    """

    if not seq or len(seq) == 0:
        return np.zeros(85)  # giữ nguyên size

    length = len(seq)
    aa_counts = Counter(seq)

    # ---------------------------
    # 1. TF cho amino acid
    # ---------------------------
    aa_list = list("ACDEFGHIKLMNPQRSTVWY")
    tf_aa = np.array([aa_counts.get(a, 0) / length for a in aa_list])

    # áp dụng IDF
    if idf_aa is not None:
        idf_vec = np.array([idf_aa.get(a, 1.0) for a in aa_list])
        tfidf_aa = tf_aa * idf_vec
    else:
        tfidf_aa = tf_aa

    # ---------------------------
    # 2. Hydrophobic, charged, length, molecular weight (k TF-IDF)
    # ---------------------------
    hydrophobic = sum(aa_counts.get(a, 0) for a in 'AILMFWYV') / length
    charged     = sum(aa_counts.get(a, 0) for a in 'DEKR') / length

    aa_weights = {'A': 89, 'C': 121, 'D': 133, 'E': 147, 'F': 165, 'G': 75, 'H': 155, 'I': 131, 'K': 146, 'L': 131, 'M': 149, 'N': 132, 'P': 115, 'Q': 146, 'R': 174, 'S': 105, 'T': 119, 'V': 117, 'W': 204, 'Y': 181}

    mol_weight = sum(aa_counts.get(a, 0) * aa_weights.get(a, 0) for a in aa_counts)

    basic_features = np.array([
        np.log1p(length),   # không TF-IDF
        hydrophobic,
        charged,
        np.log1p(mol_weight)
    ])

    # ---------------------------
    # 3. CTD groups (TF-IDF không hợp lý) → giữ nguyên TF
    # ---------------------------
    groups = {'hydrophobic': set('AILMFWYV'),'polar': set('STNQ'),'positive': set('RK'),'negative': set('DE'),'aromatic': set('FWY'),'aliphatic': set('ILV'),'small': set('ACDGNPSTV')}

    ctd_features = np.array([
        sum(1 for aa in seq if aa in g) / length for g in groups.values()
    ])

    # ---------------------------
    # 4. TF-IDF cho dipeptides
    # ---------------------------
    top_di = ['AL', 'LA', 'LE', 'EA', 'AA', 'AS', 'SA', 'EL', 'LL', 'AE', 'SE', 'ES', 'GA', 'AG', 'VA', 'AV', 'LV', 'VL', 'LS', 'SL']

    di_counts = Counter(seq[i:i+2] for i in range(length - 1))
    denom_di = max(length - 1, 1)

    tf_di = np.array([di_counts.get(dp, 0) / denom_di for dp in top_di])

    if idf_di is not None:
        idf_vec_di = np.array([idf_di.get(dp, 1.0) for dp in top_di])
        tfidf_di = tf_di * idf_vec_di
    else:
        tfidf_di = tf_di

    # ---------------------------
    # 5. TF-IDF cho tripeptides
    # ---------------------------
    top_tri = ['ALA', 'LEA', 'EAL', 'LAL', 'AAA', 'LLE', 'ELE', 'ALE', 'GAL', 'ASA', 'VLA', 'LAV', 'SLS', 'LSL', 'GLA', 'LAG', 'AVL', 'VLA', 'SLE', 'LES']

    tri_counts = Counter(seq[i:i+3] for i in range(length - 2))
    denom_tri = max(length - 2, 1)

    tf_tri = np.array([tri_counts.get(tp, 0) / denom_tri for tp in top_tri])

    if idf_tri is not None:
        idf_vec_tri = np.array([idf_tri.get(tp, 1.0) for tp in top_tri])
        tfidf_tri = tf_tri * idf_vec_tri
    else:
        tfidf_tri = tf_tri

    # ---------------------------
    # KẾT QUẢ CUỐI: TF-IDF + feature thường
    # ---------------------------
    return np.concatenate([
        tfidf_aa,
        basic_features,
        ctd_features,
        tfidf_di,
        tfidf_tri
    ])

In [37]:
# Giống y như trong feature extractor
AA_LIST = list("ACDEFGHIKLMNPQRSTVWY")

TOP_DI = ['AL', 'LA', 'LE', 'EA', 'AA', 'AS', 'SA', 'EL', 'LL', 'AE', 'SE', 'ES', 'GA', 'AG', 'VA', 'AV', 'LV', 'VL', 'LS', 'SL']

TOP_TRI = ['ALA', 'LEA', 'EAL', 'LAL', 'AAA', 'LLE', 'ELE', 'ALE', 'GAL', 'ASA', 'VLA', 'LAV', 'SLS', 'LSL', 'GLA', 'LAG', 'AVL', 'VLA', 'SLE', 'LES']


def compute_idf_aa(sequences):
    df = Counter()
    n = len(sequences)

    for seq in sequences:
        unique_aa = set(seq)
        for aa in unique_aa:
            if aa in AA_LIST:
                df[aa] += 1

    idf = {aa: math.log((n + 1) / (df[aa] + 1)) + 1 for aa in AA_LIST}
    return idf


def compute_idf_di(sequences):
    df = Counter()
    n = len(sequences)

    for seq in sequences:
        length = len(seq)
        tokens = set(
            seq[i:i+2] for i in range(length - 1)
        )
        for dp in tokens:
            if dp in TOP_DI:
                df[dp] += 1

    idf = {dp: math.log((n + 1) / (df[dp] + 1)) + 1 for dp in TOP_DI}
    return idf


def compute_idf_tri(sequences):
    df = Counter()
    n = len(sequences)

    for seq in sequences:
        length = len(seq)
        tokens = set(
            seq[i:i+3] for i in range(length - 2)
        )
        for tp in tokens:
            if tp in TOP_TRI:
                df[tp] += 1

    idf = {tp: math.log((n + 1) / (df[tp] + 1)) + 1 for tp in TOP_TRI}
    return idf


# -------------------
# GỘP LẠI TIỆN DÙNG
# -------------------
def compute_all_idf(sequences):
    return {
        "idf_aa": compute_idf_aa(sequences),
        "idf_di": compute_idf_di(sequences),
        "idf_tri": compute_idf_tri(sequences),
    }

def build_feature_matrix(seqs, idf_aa, idf_di, idf_tri):
    return np.array([extract_sequence_features_tf_idf(seq, idf_aa, idf_di, idf_tri) for seq in seqs])


In [38]:

idf_all = compute_all_idf(train_df["seq"])

idf_aa  = idf_all["idf_aa"]
idf_di  = idf_all["idf_di"]
idf_tri = idf_all["idf_tri"]
X_train = build_feature_matrix(train_df["seq"], idf_aa, idf_di, idf_tri)

In [39]:
mlb_full = MultiLabelBinarizer(sparse_output=True)
mlb_full.fit(train_df["answer"])   # fit on full set of labels present in train
n_full_labels = len(mlb_full.classes_)
print("Full labels:", n_full_labels)

# Get full binary matrix for train (we'll use it to extract reduced matrices)
y_full = mlb_full.transform(train_df["answer"])  # shape (n_train, n_full_labels)

Full labels: 26125


In [40]:
label_to_idx = {lbl: i for i, lbl in enumerate(mlb_full.classes_)}
rf_mask = np.zeros(n_full_labels, dtype=bool)
for lbl in topK:
    if lbl in label_to_idx:
        rf_mask[label_to_idx[lbl]] = True

In [41]:
A_idx = []
B_idx = []
for i, terms in enumerate(train_df["answer"]):
    if any(t in topK for t in terms):
        A_idx.append(i)
    else:
        B_idx.append(i)

print("Samples with top-K labels (A_idx):", len(A_idx))
print("Other samples (B_idx):", len(B_idx))

Samples with top-K labels (A_idx): 70698
Other samples (B_idx): 11706


In [42]:
y_rf_full = y_full[:, rf_mask]    # shape (n_train, n_rf_labels)
n_rf_labels = y_rf_full.shape[1]
print("RF will train on labels:", n_rf_labels)


RF will train on labels: 300


In [43]:
sum_in_B = np.asarray(y_full[B_idx].sum(axis=0)).reshape(-1)  # length n_full_labels
lr_mask = sum_in_B > 0   # only labels that occur at least once in B_idx

y_lr_full = y_full[:, lr_mask]
n_lr_labels = y_lr_full.shape[1]
print("LR will train on labels:", n_lr_labels)

LR will train on labels: 9367


In [44]:
X_train_full = X_train  # alias

In [45]:
X_rf_X = X_train_full[A_idx]
X_lr_X = X_train_full[B_idx] if len(B_idx) > 0 else np.empty((0, X_train_full.shape[1]))

y_rf_train = y_rf_full[A_idx]   # binary matrix for RF training (only top-K columns)
y_lr_train = y_lr_full[B_idx] if len(B_idx) > 0 else np.empty((0, y_lr_full.shape[1]))

In [46]:
# len(X_rf_X), X_rf_X[0], X_rf_X[0].shape
# len(y_rf_train), y_rf_train[0]
# len(y_rf_train), y_rf_train[0], y_rf_train[0].shape

In [47]:
clf_rf = OneVsRestClassifier(
    HistGradientBoostingClassifier(max_iter=300, random_state=42),
    n_jobs=-1,
    verbose=1
)

print("Training RF on top-K labels ...")
clf_rf.fit(X_rf_X, y_rf_train)
print("RF training done.")

Training RF on top-K labels ...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
clf_lr = OneVsRestClassifier(
    LogisticRegression(random_state=42, max_iter=500),
    n_jobs=-1,
    verbose=1
)

print("Training LR on remaining labels ...")
clf_lr.fit(X_lr_X, y_lr_train)
print("LR training done.")

In [ ]:
X_test = build_feature_matrix(test_df["seq"], idf_aa, idf_di, idf_tri)
# X_test = build_feature_matrix_v2(test_df["seq"])

n_test = X_test.shape[0]
arr_ids = test_df["id"].values

In [ ]:
rf_positions = np.where(rf_mask)[0]
lr_positions = np.where(lr_mask)[0]

In [ ]:
batch_size = 7182
threshold = 0.02

rf_weight_top = 0.9
rf_weight_else = 0.3

class_weight = np.array([rf_weight_top if lbl in topK else rf_weight_else for lbl in mlb_full.classes_], dtype=float)

all_probs = []

for start in range(0, n_test, batch_size):
    end = min(start + batch_size, n_test)
    X_batch = X_test[start:end]
    print(f"Predicting batch {start}:{end} ...")

    # Get reduced probabilities
    prob_rf_reduced = clf_rf.predict_proba(X_batch)  # shape (batch, n_rf_labels)
    prob_lr_reduced = clf_lr.predict_proba(X_batch)  # shape (batch, n_lr_labels)

    # Expand to full-length probability vectors
    prob_rf_full = np.zeros((X_batch.shape[0], n_full_labels), dtype=float)
    prob_lr_full = np.zeros((X_batch.shape[0], n_full_labels), dtype=float)

    # assign reduced outputs to their positions in full vector
    if prob_rf_reduced.shape[1] > 0:
        prob_rf_full[:, rf_positions] = prob_rf_reduced

    if prob_lr_reduced.shape[1] > 0:
        prob_lr_full[:, lr_positions] = prob_lr_reduced

    # ---------- 8. dynamic class-wise weight ----------
    # class_weight: for labels in topK -> rf_weight_top, else rf_weight_else
    

    # final probability
    prob_final = np.zeros((X_batch.shape[0], n_full_labels), dtype=float)

    for lbl_idx in range(n_full_labels):
        rf_has = rf_mask[lbl_idx]          # RF được train nhãn này?
        lr_has = lr_mask[lbl_idx]          # LR được train nhãn này?
    
        if rf_has and lr_has:
            # nhãn có trong cả RF và LR → dùng weighted
            w = rf_weight_top if mlb_full.classes_[lbl_idx] in topK else rf_weight_else
            prob_final[:, lbl_idx] = w * prob_rf_full[:, lbl_idx] + (1 - w) * prob_lr_full[:, lbl_idx]
    
        elif rf_has:
            # nhãn chỉ có trong RF → giữ nguyên  
            prob_final[:, lbl_idx] = prob_rf_full[:, lbl_idx]
    
        elif lr_has:
            # nhãn chỉ có trong LR → giữ nguyên  
            prob_final[:, lbl_idx] = prob_lr_full[:, lbl_idx]
    
        else:
            # nhãn không train ở RF và LR (cực hiếm): cho 0
            prob_final[:, lbl_idx] = 0.0

    # ---------- 9. threshold and collect output ----------
    for i in range(X_batch.shape[0]):
        vec = prob_final[i]
        preds = []
        # filter by threshold
        above = np.where(vec >= threshold)[0]
        for idx in above:
            preds.append((mlb_full.classes_[idx], float(vec[idx])))
        all_probs.append({"id": arr_ids[start + i], "preds": preds})

print("Prediction finished. Total:", len(all_probs))

In [ ]:
all_probs[:20]

In [ ]:
output_file = "submission.tsv"

with open(output_file, "w") as f:
    for mp in all_probs:
        # mp['id'], mp['pred']
        for pred, prob in mp['preds']:
            f.write(mp['id']+"\t"+pred+"\t"+str(round(prob, 3))+"\n")